# Importing Libraries

In [61]:
!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install optuna
!pip install -U "neptune[optuna]"

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [62]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel,AutoTokenizer,AdamW
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import random
from torchmetrics.classification import BinaryAccuracy,MulticlassF1Score,MulticlassPrecision,MulticlassRecall,MulticlassAccuracy,BinaryConfusionMatrix
from torchmetrics.collections import MetricCollection
import optuna
from google.colab import userdata
import neptune
import uuid
import neptune.integrations.optuna as npt_utils
import  hashlib
import time
import math
import pickle

In [63]:
from sklearn.model_selection import StratifiedKFold

In [64]:
from torch.utils.data import SubsetRandomSampler

# Setting up the GPU or CPU

In [65]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NEPTUNE_API_TOKEN"] = userdata.get('NEPTUNE_API_TOKEN')
device = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

# Define Variable Values

In [66]:
# @title Define Transformer Model Name
bert_model_name = "cardiffnlp/twitter-roberta-base-sep2022" # @param {type:"string"}

In [67]:
# @title Define Hugging Face Dataset Name
dataset_name = "krishan-CSE/Davidson_Hate_Speeh_and_Emotion" # @param {type:"string"}



In [68]:
# @title Define Neptuna Project Name
project_name='FYP-DCL/fyp-thushalya'# @param {type:"string"}
neptune_id='' # @param {type:"string"} #if you want to start a new study keep it blank

In [69]:
# @title Define Neptuna DCL Project Name & Study Id
dcl_study_id='DCLNEWDV-480' # @param {type:"string"}
best_trial_id='DCLNEWDV-608' # @param {type:"string"}
dcl_project_name='pasansk/DCL-NEWDV'# @param {type:"string"}

In [70]:
# @title Define Experiment Name
study_name = "Emotion" # @param {type:"string"}



In [71]:
dataset_tag = dataset_name
model_tag = bert_model_name

##Setting up Neptuna

In [72]:
def load_optuna_states(run:neptune.Run,directory:str):
  run[f'{directory}/optuna_storage'].download()
  run[f'{directory}/sampler'].download()
  run[f'{directory}/pruner'].download()
  run.wait()

In [73]:
if neptune_id=='':
  study_id=str(hashlib.md5(str(time.time()).encode()).hexdigest()[:8])
  sweep_id = uuid.uuid1()
  run_study_level = neptune.init_run(project=project_name,capture_stdout=True,capture_stderr=True,capture_traceback=True, capture_hardware_metrics=True,tags=[dataset_tag , "hyper-parameter-tuning", model_tag],name=(study_name+'_'+study_id))
  run_study_level["sweep-id"] = str(sweep_id)
  run_study_level["sys/tags"].add("study-level")
else:
  run_study_level = neptune.init_run(with_id=neptune_id,project=project_name,capture_stdout=True,capture_stderr=True,capture_traceback=True, capture_hardware_metrics=True)
  name=run_study_level["sys/name"].fetch()
  run_study_level.wait()
  study_name,study_id = tuple(name.rsplit('_', 1))
  sweep_id=run_study_level["sweep-id"].fetch()
  load_optuna_states(run=run_study_level,directory="optuna")

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/FYP-DCL/fyp-thushalya/e/FYPTHUS-1697


In [74]:
#@title Define Transformer Model Tokenizer Max Padding Length
SEED = 1234
PADDING_MAX_LENGTH = 45 # @param {type:"integer"}


# Setting Random Seed for Reproducibility

In [75]:
def setup_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [76]:
setup_seed(SEED)

# Loading Train,Dev Datasets

In [77]:
data_files = {"train":"train.csv"}
dataset = load_dataset(dataset_name,data_files=data_files)

# Loading the Tokernizer for the Transformer Model

In [78]:
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

##Define the Tokenizer Function

In [79]:
def tokenize_function(examples):
    return tokenizer.batch_encode_plus(examples["text"], padding='max_length',max_length=PADDING_MAX_LENGTH,add_special_tokens=True,truncation=True)

## Tokenize the Dataset

In [80]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 18586
    })
})

In [81]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

## Merge Author coloumns to a vector

In [82]:
# Define a function to merge columns into a list
def merge_emotion_columns_to_list(example):
    return {'emotion_vector': [example['anger'], example['anticipation'], example['disgust'],example['fear'],example['joy'],example['love'],example['optimism'],example['pessimism'],example['sadness'],example['surprise'],example['trust']]}

# Apply the function to each example in the dataset
tokenized_datasets = tokenized_datasets.map(merge_emotion_columns_to_list)

In [83]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'input_ids', 'attention_mask', 'emotion_vector'],
        num_rows: 18586
    })
})

## Remove Unwanted Coloumns

In [84]:
tokenized_datasets=tokenized_datasets.remove_columns(['text','anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'])

In [85]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'emotion_vector'],
        num_rows: 18586
    })
})

## Format the coloumns

In [86]:
tokenized_datasets=tokenized_datasets.with_format("torch")

# Define the Dual Contrastive Learning Architecture



In [87]:
class DCLArchitecture(nn.Module):
    def __init__(self,dropout:float,bert_model_name:str='bert-base-cased'):
        super(DCLArchitecture, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dim = 768
        self.dense = nn.Linear(self.dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,batch_tokenized, if_train=False):
        input_ids = batch_tokenized['input_ids']
        attention_mask = batch_tokenized['attention_mask']
        bert_output = self.bert(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        bert_cls_hidden_state = bert_output[1]

        if if_train:
            bert_cls_hidden_state_aug = self.dropout(bert_cls_hidden_state)
            bert_cls_hidden_state = torch.cat((bert_cls_hidden_state, bert_cls_hidden_state_aug), dim=1).reshape(-1, self.dim)
        else:
            bert_cls_hidden_state = self.dropout(bert_cls_hidden_state)

        linear_output = self.dense(bert_cls_hidden_state)
        linear_output = linear_output.squeeze(1)

        return bert_cls_hidden_state, linear_output

## Define Focal Loss

In [88]:
class FocalLoss(nn.Module):
    def __init__(self, alpha:float=0.4, gamma:float=2, size_average:bool=True):
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor(alpha)
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, pred, target):

        device = target.device
        self.alpha = self.alpha.to(device)

        pred = nn.Sigmoid()(pred)
        pred = pred.view(-1, 1)
        target = target.view(-1, 1)
        pred = torch.cat((1-pred, pred), dim=1)

        class_mask = torch.zeros(pred.shape[0], pred.shape[1]).to(device)
        class_mask.scatter_(1, target.view(-1, 1).long(), 1.)
        probs = (pred * class_mask).sum(dim=1).view(-1, 1)
        probs = probs.clamp(min=0.0001, max=1.0)

        log_p = probs.log()
        alpha = torch.ones(pred.shape[0], pred.shape[1]).to(device)
        alpha[:, 0] = alpha[:, 0] * (1 - self.alpha)
        alpha[:, 1] = alpha[:, 1] * self.alpha
        alpha = (alpha * class_mask).sum(dim=1).view(-1, 1)

        batch_loss = -alpha * (torch.pow((1 - probs), self.gamma)) * log_p

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()

        return loss

# Define the Extended DCL Architecture with Authers

In [89]:
class EmotionGuidedDCLModel(nn.Module):
    def __init__(self,dcl_model:nn.Module,dropout:float=0.25):
        super(EmotionGuidedDCLModel, self).__init__()
        self.dcl_model = dcl_model
        self.dim = 779
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.dim, 1)
        # Freeze all layers
        for param in self.dcl_model.parameters():
            param.requires_grad = False

    def forward(self,batch_tokenized):
        input_ids = batch_tokenized['input_ids']
        attention_mask = batch_tokenized['attention_mask']
        emotion_vector = batch_tokenized['emotion_vector']
        bert_output = self.dcl_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        bert_cls_hidden_state = bert_output[1]
        combined_vector =torch.cat((bert_cls_hidden_state,emotion_vector), 1)
        d_combined_vector=self.dropout(combined_vector)
        linear_output = self.linear(d_combined_vector)
        pred_linear = linear_output.squeeze(1)
        return pred_linear

# Configuring the DCL Model

## Load the Best Trial Checkpoint

In [90]:
run_dcl_study = neptune.init_run(with_id=dcl_study_id,project=dcl_project_name,mode='read-only')

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/pasansk/DCL-NEWDV/e/DCLNEWDV-480


In [91]:
run_best_model_trial = neptune.init_run(with_id=best_trial_id,project=dcl_project_name,mode="read-only")

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/pasansk/DCL-NEWDV/e/DCLNEWDV-608


In [92]:
best_prams=run_best_model_trial["parameters"].fetch()

In [93]:
def load_checkpoint(run: neptune.Run,check_point_name:str):
    model_ext = run[check_point_name]["model"].fetch_extension()
    run[check_point_name]["model"].download()  # Download the checkpoint
    run.wait()
    # Load the checkpoint
    checkpoint = {
        "model_state_dict":torch.load(f"model.{model_ext}"),
    }
    return checkpoint

In [94]:
checkpoint=load_checkpoint(run=run_dcl_study,check_point_name="model_checkpoints/")

Fetching file...: 0 [00:00, ?/s]

In [95]:
dcl_model = DCLArchitecture(bert_model_name=bert_model_name,dropout=best_prams["DROPOUT"])
dcl_model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sep2022 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DCLArchitecture(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

In [96]:
dcl_model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [97]:
run_dcl_study.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/pasansk/DCL-NEWDV/e/DCLNEWDV-480/metadata


In [98]:
run_best_model_trial.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/pasansk/DCL-NEWDV/e/DCLNEWDV-608/metadata


In [99]:
fined_tuned_bert_model=dcl_model.bert

# Fine Tuning the Model

In [100]:
optuna_storage_location="content/optuna_storage.db"

## Format the Labels

In [101]:
def is_supported_values(value):
  if not math.isinf(value) and not math.isnan(value):
    return True
  else:
    return False

## Define Training Loop

In [102]:
def training_loop(model, train_dataloader,optimizer,criteon,device,fold:int,run_trial_level):
    accuracy_metric = BinaryAccuracy()
    accuracy_metric.to(device)
    progress_bar = tqdm(range(len(train_dataloader)))
    model.train()
    total_train_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"]
        pred = model(batch)
        loss = criteon(pred, labels.float())
        total_train_loss += loss.item()

        loss_value=loss.item()
        if is_supported_values(loss_value):
          run_trial_level[f"fold_{fold}/train/batch/loss"].append(loss_value)

        pred_sig = torch.sigmoid(pred)

        preds_detach=torch.round(pred_sig.detach())

        batch_level_accuracy=accuracy_metric(preds_detach,labels)

        if is_supported_values(batch_level_accuracy.item()):
          run_trial_level[f"fold_{fold}/train/batch/accuracy"].append(batch_level_accuracy.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.update(1)
    average_epoch_train_loss = total_train_loss / len(train_dataloader)  # Compute average epoch loss
    train_accuracy =  accuracy_metric.compute()
    accuracy_metric.reset()
    return average_epoch_train_loss,train_accuracy


## Define Validation Loop

In [103]:
def validation_loop(model, dev_dataloader,criteon, device,average='macro'):
    collection_metric = MetricCollection(
          BinaryAccuracy(),
          MulticlassPrecision(num_classes=2,average=average),
          MulticlassRecall(num_classes=2,average=average),
          MulticlassF1Score(num_classes=2,average=average),
          BinaryConfusionMatrix()
    )
    collection_metric.to(device)
    model.eval()
    total_dev_loss = 0.0
    for batch in dev_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"]

        with torch.no_grad():
            pred = model(batch)
            loss = criteon(pred, labels.float())
            pred_sig = torch.round(torch.sigmoid(pred))
        nan_exist = torch.isnan(pred_sig).any().item()
        if nan_exist:
          collection_metric.reset()
          return None
        total_dev_loss += loss.item()
        collection_metric.update(pred_sig,labels)
    average_epoch_dev_loss = total_dev_loss / len(dev_dataloader)  # Compute average epoch loss
    result = collection_metric.compute()
    result["Loss"]=average_epoch_dev_loss
    result["BinaryConfusionMatrix"],_=collection_metric.BinaryConfusionMatrix.plot()
    collection_metric.reset()
    return result

## Define Test Loop

## Hyperparameter Tuning

In [104]:
storage = optuna.storages.RDBStorage("sqlite:////"+optuna_storage_location)

In [105]:
def define_model(dcl_model):
    DROPOUT = 0.5
    model = EmotionGuidedDCLModel(dcl_model=dcl_model,dropout=DROPOUT)
    model.to(device)
    return model

In [106]:
def get_data_loaders(tokenized_datasets,train_sampler,dev_sampler,batch_size):
    BATCH_SIZE = batch_size
    train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=BATCH_SIZE, sampler=train_sampler)
    dev_dataloader = DataLoader(tokenized_datasets['train'], batch_size=BATCH_SIZE, sampler=dev_sampler)
    return train_dataloader,dev_dataloader

In [107]:
def save_optuna_states():
  with open("sampler.pkl", "wb") as fout:
    pickle.dump(study.sampler, fout)
  with open("pruner.pkl", "wb") as fout:
    pickle.dump(study.pruner, fout)

In [108]:
def upload_optuna_states(run:neptune.Run,directory:str):
    run[f"{directory}/sampler"].upload('sampler.pkl')
    run[f"{directory}/pruner"].upload('pruner.pkl')
    run[f"{directory}/optuna_storage"].upload("optuna_storage.db")
    run.wait()

In [109]:
import matplotlib.pyplot as plt

In [110]:
def upload_metrics_result(directory:str,result:dict,run: neptune.Run):
    run[f"{directory}/Accuracy"]=result["BinaryAccuracy"].item()
    run[f"{directory}/Focal_Loss"]=result["Loss"]
    run[f"{directory}/Weighted_Precision"]=result["MulticlassPrecision"].item()
    run[f"{directory}/Weighted_Recall"]=result["MulticlassRecall"].item()
    run[f"{directory}/Weighted_F1Score"]=result["MulticlassF1Score"].item()
    run[f"{directory}/ConfusionMatrix"].upload(result["BinaryConfusionMatrix"])
    run.wait()
    plt.close()

In [111]:
def upload_metrics_series_result(directory:str,result:dict,run: neptune.Run):
    run[f"{directory}/Accuracy"].append(result["BinaryAccuracy"].item())
    run[f"{directory}/Focal_Loss"].append(result["Loss"])
    run[f"{directory}/Weighted_Precision"].append(result["MulticlassPrecision"].item())
    run[f"{directory}/Weighted_Recall"].append(result["MulticlassRecall"].item())
    run[f"{directory}/Weighted_F1Score"].append(result["MulticlassF1Score"].item())
    run.wait()

In [112]:
def objective(train_sampler,dev_sampler,param,fold,run_trial_level):
    setup_seed(param["SEED"])
    train_dataloader,dev_dataloader =get_data_loaders(tokenized_datasets=tokenized_datasets,train_sampler=train_sampler,dev_sampler=dev_sampler,batch_size=param["BATCH_SIZE"])
    model = define_model(fined_tuned_bert_model)
    optimizer = torch.optim.AdamW(model.parameters(),lr = param["LEARNING_RATE"])
    criteon = FocalLoss(param["ALPHA"], param["GAMMA"])

    for epoch in range(param["EPOCHS"]):
        average_epoch_train_loss,train_accuracy = training_loop(model=model, train_dataloader=train_dataloader,optimizer=optimizer,criteon=criteon,device=device,fold=fold,run_trial_level=run_trial_level)
        dev_result_metrics = validation_loop(model=model, dev_dataloader=dev_dataloader,criteon=criteon, device=device,average='weighted')

        if  dev_result_metrics==None:
              return -1,None
        else:
            objective_value = dev_result_metrics['MulticlassF1Score'].item()

        run_trial_level[f"fold_{fold}/train/epoch/loss"].append(average_epoch_train_loss)
        run_trial_level[f"fold_{fold}/train/epoch/accuracy"].append(train_accuracy.item())

        upload_metrics_series_result(directory=f"fold_{fold}/evaluation/epoch",result=dev_result_metrics,run=run_trial_level)

    run_trial_level[f"fold_{fold}/train/loss"]=average_epoch_train_loss
    run_trial_level[f"fold_{fold}/train/accuracy"]=train_accuracy.item()

    upload_metrics_result(directory=f"fold_{fold}/evaluation",result=dev_result_metrics,run=run_trial_level)
    result=dev_result_metrics
    result["train_accuracy"]=train_accuracy
    result["train_loss"]=average_epoch_train_loss
    return objective_value,result


In [113]:
from statistics import mean

In [114]:
def objective_cv(trial):
    # Define the hyperparameters to optimize
    BATCH_SIZE = trial.suggest_int("batch_size",100,256)
    EPOCHS = trial.suggest_int("epochs",1,2)
    ALPHA= trial.suggest_float("alpha", 0.4,1.0,log=True)
    GAMMA = trial.suggest_float("gamma", 0.1,5.0)
    LR = trial.suggest_float("learning_rate", 1e-5 ,1e-3,log=True)
    param = {
        "EPOCHS": EPOCHS,
        "ALPHA": ALPHA,
        "GAMMA": GAMMA,
        "BATCH_SIZE":BATCH_SIZE,
        "LEARNING_RATE":LR,
        "DROPOUT":0.5,
        "SEED":SEED
    }

     # Create a trial-level run
    run_trial_level = neptune.init_run(project=project_name,capture_stdout=True,capture_stderr=True,capture_traceback=True, capture_hardware_metrics=True,
    name=study_name+"_"+str(study_id)+"_"+str(trial.number),tags=[model_tag,dataset_tag,(study_name+'_'+study_id),str(trial.number+1)])

    # Log sweep ID to trial-level run
    run_trial_level["sys/tags"].add("trial-level")
    run_trial_level["sweep-id"] = str(sweep_id)

    # Log parameters of a trial-level run
    run_trial_level["parameters"] = param

    SK_Fold = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True)
    g_cpu = torch.Generator()
    g_cpu.manual_seed(SEED)
    train_accuracy = []
    train_loss = []
    dev_accuracy = []
    dev_loss = []
    dev_f1score = []
    dev_recall = []
    dev_precision = []
    for fold, (train_indices, dev_indices) in enumerate(SK_Fold.split(tokenized_datasets['train'],tokenized_datasets['train']['labels'])):

        train_sampler = SubsetRandomSampler(train_indices,g_cpu)
        dev_sampler = SubsetRandomSampler(dev_indices,g_cpu)
        objective_value,result = objective(train_sampler=train_sampler, dev_sampler=dev_sampler,param=param,fold=(fold+1),run_trial_level=run_trial_level)
        trial.report(objective_value, (fold+1))
        if trial.should_prune():
            run_trial_level["sys/tags"].add("pruned")
            run_trial_level.stop()
            raise optuna.exceptions.TrialPruned()
        train_accuracy.append(result["train_accuracy"].item())
        dev_accuracy.append(result["BinaryAccuracy"].item())
        train_loss.append(result["train_loss"])
        dev_loss.append(result["Loss"])
        dev_f1score.append(result["MulticlassF1Score"].item())
        dev_recall.append(result["MulticlassRecall"].item())
        dev_precision.append(result["MulticlassPrecision"].item())


    objective_value = mean(dev_f1score)
    run_trial_level["global/metrics/train/mean_accuracy"] = mean(train_accuracy)
    run_trial_level["global/metrics/train/mean_loss"] = mean(train_loss)
    run_trial_level["global/metrics/evaluation/mean_accuracy"] = mean(dev_accuracy)
    run_trial_level["global/metrics/evaluation/mean_loss"] = mean(dev_loss)
    run_trial_level["global/metrics/evaluation/mean_weighted_f1score"] = objective_value
    run_trial_level["global/metrics/evaluation/mean_weighted_Recall"] = mean(dev_recall)
    run_trial_level["global/metrics/evaluation/mean_weighted_Precision"] = mean(dev_precision)

    run_trial_level.stop()
    save_optuna_states()
    upload_optuna_states(run=run_study_level,directory="optuna")
    return objective_value

In [115]:
if neptune_id=='':
    sampler = optuna.samplers.TPESampler(seed=SEED,multivariate=True,group=True,n_startup_trials=20,consider_prior=True)
    pruner=optuna.pruners.ThresholdPruner(lower=0)
else:
    sampler=pickle.load(open("sampler.pkl", "rb"))
    pruner=pickle.load(open("pruner.pkl", "rb"))

/usr/local/lib/python3.10/dist-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.

/usr/local/lib/python3.10/dist-packages/optuna/samplers/_tpe/sampler.py:330: ExperimentalWarning:

``group`` option is an experimental feature. The interface can change in the future.



In [116]:
neptune_callback = npt_utils.NeptuneCallback(run_study_level,target_names=["weighted_F1_Score"])

In [117]:
study = optuna.create_study(study_name=study_name,storage=storage,load_if_exists=True,direction='maximize',sampler=sampler,pruner=pruner)

[I 2024-03-14 10:48:44,938] Using an existing study with name 'Emotion' instead of creating a new one.


In [ ]:
study.optimize(objective_cv, n_trials=120,callbacks=[neptune_callback])

trial = study.best_trial

print('Best Value: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/FYP-DCL/fyp-thushalya/e/FYPTHUS-1698


  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

...

[I 2024-03-14 12:21:11,053] Trial 250 finished with value: 0.9345213294029235 and parameters: {'batch_size': 135, 'epochs': 2, 'alpha': 0.7270544607839476, 'gamma': 2.0653771514399994, 'learning_rate': 0.0006004223649623664}. Best is trial 174 with value: 0.9382787227630616.


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/FYP-DCL/fyp-thushalya/e/FYPTHUS-1709


  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

In [ ]:
save_optuna_states()
run_study_level["optuna/sampler"].upload('sampler.pkl')
run_study_level["optuna/pruner"].upload('pruner.pkl')
run_study_level["optuna/optuna_storage"].upload("optuna_storage.db")

In [ ]:
run_study_level.stop()